# Projeto 2 - Ciência dos Dados

**Nome**: Jonas da Silva Lopes

**Nome**: William Silva


# Classificador automático de sentimento

## Preparando o ambiente no jupyter:

In [35]:
%%capture
!pip install tweepy
!pip install emoji

In [36]:
import tweepy
import emoji
import math
import os.path
import pandas as pd
import json
import re 
import functools
import operator
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@William48253649***

In [37]:
#Identificador da conta no twitter: @William48253649

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [38]:
#Produto escolhido
produto = 'Nubank'

#Quantidade mínima de mensagens capturadas
n = 500
#Quantidade mínima de mensagens para a base de treinamento
t = 300

#Filtro de língua
lang = 'pt'

Capturando os dados do twitter:

In [39]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower()) #Lower para deixar tudo minúsculo e facilitar a comparação
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [40]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])}).set()
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])}).set()
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Após realizar a classificação manual das mensagens, como irrelevante (0) ou relevante (1), partimos para  a mudança desses valores para algo mais palpável para a análise. Com isso, fazemos a alteração que pode ser vista abaixo.

Mas, para realizar isso, primeiro foi necessário determinar critérios para a classificação:

- Mencionar o produto;
- A menção ao produto deve ser acompanhada de uma opinião;
- A opinião pode ser demonstrada na forma de indagações, reclamações, pode envolver sarcasmo, elogios e sugestões sobre serviços;
- A opinião afirmada deve ser clara;
- Emoctions também representam opiniões.

In [323]:
mensagens = pd.read_excel("Nubank.xlsx")
mensagens.Relevância = mensagens.Relevância.astype('category')
mensagens.Relevância.cat.categories = ('Irrelevante', 'Relevante')

print("A quantidade de cada cada um é: \n\n", mensagens.Relevância.value_counts())

relevante = mensagens[mensagens.Relevância=="Relevante"]
irrelevante = mensagens[mensagens.Relevância=="Irrelevante"]

A quantidade de cada cada um é: 

 Irrelevante    184
Relevante      116
Name: Relevância, dtype: int64


In [59]:
'''A Partir dos dados obtidos, nota-se que:
        p(relevante) = 116/300
        p(irrelevante) = 184/300
'''
p_relev = 116/300*100
print("A probabilidade de ser relevante é", p_relev, "%")

p_irrelev = 184/300*100
print("A probabilidade de ser irrelevante é", p_irrelev, "%")

A probabilidade de ser relevante é 38.666666666666664 %
A probabilidade de ser irrelevante é 61.33333333333333 %


In [329]:
''' Função que troca pontuação por espaço '''
def cleanup(text):
    punctuation = '[!\-.:?;/,|@"\'()]'
    pattern = re.compile(punctuation)
    # Abaixo, determina que se troca por espaço
    text_subbed = re.sub(pattern, ' ', text)
    
    emoji_dividir = emoji.get_emoji_regexp().split(text_subbed)
    espaco_dividir = [substr.split() for substr in emoji_dividir]
    split = functools.reduce(operator.concat, espaco_dividir)
    
    return split

# Usando a função apply para fazer a limpeza nas mensagens
nubank_relev = relevante.Treinamento.apply(cleanup)
nubank_irrelev = irrelevante.Treinamento.apply(cleanup)

# Pegando as palavras da lista de Relevantes para contar
def pega_text(texto):
    # Pega as palavras para colocar numa lista
    lista = []
    i = 0
    for linha in texto:
        if len(linha) > 1:
            while i < len(linha):
                lista.append(linha[i])
                i += 1  
            i = 0
        elif len(linha) != 1 and (len(linha)-1) != 1:
            a = texto
            return a 
    return lista

relevante.Treinamento

0      rindo mas preocupada pois cada dia parece que ...
6          eu amo o nubank mesmo https://t.co/rrpb89yes9
7      meu pai pediu um cartão novo pra mim pra pegar...
8          quem inventou o nubank nem é gente, é um anjo
11     @castrocastrado @nubank se quiser eu te indico...
12     @1masterball @haunted_electra me assusta muito...
13     prejuízo nubank: banco tem 12 milhões de clien...
15     @marialmeida93 @thaioliv_ @nubank menina que t...
19     @nubank @itoncrf_ e assim mesmo n me aprovaram...
21          depois de anos tentando,fui aceita no nubank
22     rt @afropreticinha: aff coma assim o cartão da...
25     fiquei assustada quando vi que minha fatura do...
26     nubank nunca me liberou crédito, agr q libera....
27               meu nubank chegou😍 vos declaro falência
28     @lucasafoliveira @nubank ele trava assim e não...
31     sério, se eu perder minha bolsa por conta do n...
32        eu toda com meu nubank https://t.co/4av08gusbn
33     querido @nubank, a gente

### Frequência Absoluta

Relevantes

In [332]:
# Frequência dos Relevantes
lista_nubank_relev = pd.DataFrame(pega_text(nubank_relev))
lista_nubank_relev[0].value_counts()

nubank            122
o                  58
de                 54
e                  52
eu                 47
que                45
a                  35
meu                34
me                 29
co                 28
t                  28
https              28
um                 27
não                26
é                  25
do                 23
cartão             23
no                 20
uma                16
pra                15
na                 15
q                  15
to                 14
com                14
só                 13
mais               12
limite             12
tem                11
da                 11
nunca              11
                 ... 
robertoper3ir4      1
forças              1
anoja               1
paciência           1
outro               1
enxoval             1
pianoetlux          1
quiser              1
sempre              1
6                   1
tira                1
😎                   1
vazou               1
dado                1
cfo       

Irrelevantes

In [295]:
# Frequência dos Irrelevantes
lista_nubank_irrelev = pd.DataFrame(pega_text(nubank_irrelev))
lista_nubank_irrelev[0].value_counts()

nubank             154
o                   71
de                  67
e                   62
que                 60
co                  58
https               58
t                   58
a                   57
não                 52
é                   38
um                  36
eu                  34
com                 33
da                  33
do                  31
se                  30
gente               29
tem                 28
rt                  28
meu                 26
pra                 26
em                  25
no                  25
cartão              24
você                24
na                  24
só                  22
uma                 22
conta               22
                  ... 
serve                1
rolando              1
interferir           1
minhas               1
zq6cca3lv1           1
hahahaa              1
castrocastrado       1
consiga              1
pagou                1
fechar               1
porra                1
acesso               1
barras     

### Frequência Relativa

Relevante

In [299]:
lista_nubank_relev[0].value_counts(True)

nubank            0.056092
o                 0.026667
de                0.024828
e                 0.023908
eu                0.021609
que               0.020690
a                 0.016092
meu               0.015632
me                0.013333
co                0.012874
t                 0.012874
https             0.012874
um                0.012414
não               0.011954
é                 0.011494
do                0.010575
cartão            0.010575
no                0.009195
uma               0.007356
pra               0.006897
na                0.006897
q                 0.006897
to                0.006437
com               0.006437
só                0.005977
mais              0.005517
limite            0.005517
tem               0.005057
da                0.005057
nunca             0.005057
                    ...   
robertoper3ir4    0.000460
forças            0.000460
anoja             0.000460
paciência         0.000460
outro             0.000460
enxoval           0.000460
p

Irrelevante

In [301]:
lista_nubank_irrelev[0].value_counts(True)

nubank             0.045441
o                  0.020950
de                 0.019770
e                  0.018294
que                0.017704
co                 0.017114
https              0.017114
t                  0.017114
a                  0.016819
não                0.015344
é                  0.011213
um                 0.010623
eu                 0.010032
com                0.009737
da                 0.009737
do                 0.009147
se                 0.008852
gente              0.008557
tem                0.008262
rt                 0.008262
meu                0.007672
pra                0.007672
em                 0.007377
no                 0.007377
cartão             0.007082
você               0.007082
na                 0.007082
só                 0.006492
uma                0.006492
conta              0.006492
                     ...   
serve              0.000295
rolando            0.000295
interferir         0.000295
minhas             0.000295
zq6cca3lv1         0

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [320]:
for linha in mensagens.Treinamento:
    lista_linha = cleanup(linha)
    '''chama a função com o atributo a'''

print(a)

['1masterball', 'renatojg', 'nossa', 'clonaram', 'um', 'nubank', '😱']


In [326]:
a = cleanup(mensagens.Treinamento)

TypeError: expected string or bytes-like object

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**